# 🤔Movie lens 내쉬균형을 이용해서 어쩌구

### context

1. Data load
2. SVD 평점 예측
3. Clustering
4. NASH aggregation
5. Measure

## 데이터 LOAD

## Library 

In [1]:
#기본 라이브러리
import pandas as pd
import numpy as np
import math

# SVD 행렬축소
from sklearn.decomposition import TruncatedSVD

# 제작 함수
import HA_ndcg

In [2]:

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')

#컬럼명 수정
ratings.columns = ['userid','movieid','rating','timestamp']

#시간 컬럼 제거
ratings = ratings.drop(['timestamp'], axis=1)

print("총 주어진 ratings 데이터 : ", ratings.shape)

#피벗테이블로 만들어서 사용자-아이템 행렬 확인하기
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])
rating_table

총 주어진 ratings 데이터 :  (100000, 3)


movieid,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userid,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Trian 데이터와 Test 데이터는 10만개의 데이터를 8만개와 2만개로 나누어 분류했다.  
이후 pivot 하여 User-item matrix를 만들었다.

In [3]:
# 80000 row
user_train = pd.read_csv('../ml-100k_GRS/u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train = user_train[['userID','itemID','rating']]
user_train = user_train.pivot_table('rating', index = 'userID',columns = 'itemID')


# 20000 row
user_test = pd.read_csv('../ml-100k_GRS/u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]
user_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID')

## 2. SVD 로 평점 예측

내쉬균형을 도출하기 위해 우선 모든 영화들에 대한 예측평점을 알고 있다고 가정했다.  
예측평점은 truncated SVD 를 이용해 구현했다.

<a href="https://data-science-hi.tistory.com/82">SVD 를 통한 평점예측</a><br>
<a herf="The Algorithm: Item-based Filtering Enhanced by SVD">The Algorithm: Item-based Filtering Enhanced by SVD</a>

1. 각 아이템 (coloumn) 별 평균을 구해 결측치를 대체
2. 대체된 행렬에 각 사용자 (row) 별 평균을 빼서 정규화
3. 특잇값 분해(SVD) 후 행렬 축소(K=12)
4. 특징을 가지고 있는 대각행렬 Sigma 으로 유사 행렬 도출
5. 2번에서 뺐던 각 사용자 (row) 별 평균을 더해서 예측평점 구현

(ref - 파이썬 머신러닝 완벽가이드 - 571p)

In [4]:
# 결측치 대체 전 각 행렬 별 평균값 계산
row_mean= list(rating_table.mean(axis=1))
col_mean= list(rating_table.mean())

print(f"5 번째 행까지의 각 사용자별 평균 :\n {row_mean[:5]}\n")
print(f"3 번째 열까지의 각 영화별 평균 : \n {col_mean[:3]}")

5 번째 행까지의 각 사용자별 평균 :
 [3.610294117647059, 3.7096774193548385, 2.7962962962962963, 4.333333333333333, 2.874285714285714]

3 번째 열까지의 각 영화별 평균 : 
 [3.8783185840707963, 3.2061068702290076, 3.033333333333333]


In [5]:

# 1. 아이템 별 평균 대체
def Rating_filled_in(rating_table):
    for col in range(len(rating_table.columns)):
        
        # 열의 평균을 구한다.
        col_num = [i for i in rating_table.iloc[:,col] if math.isnan(i)==False]
        col_mean = sum(col_num)/len(col_num)
        
        # NaN을 가진 행은 위에서 구한 평균 값으로 채워준다.
        col_update = [i if math.isnan(i)==False else col_mean for i in rating_table.iloc[:,col]]
        rating_table.iloc[:,col] = col_update
        
    return rating_table


# 2. 각 사용자 별 평균을 빼주며 정규화
def Rating_norm(rating_table):
    
    #추후에 다시 더하기 위해 각 행별 평균을 저장
    row_mean_data = []

    for row in range(len(rating_table)):
        

        # 행의 평균을 구한다.
        row_mean= sum(rating_table.iloc[row,:])/len(rating_table.iloc[row,:])
        
        # 1행부터 마지막행까지의 평균 데이터 저장  
        row_mean_data.append(row_mean)

        # 해당 행의 모든 값에 행 평균 값을 뺀다.
        row_update = [i - row_mean for i in rating_table.iloc[row,:]]
        rating_table.iloc[row,:] = row_update

    return rating_table, row_mean_data
    




In [6]:
rating_table = Rating_filled_in(rating_table)
rating_table, row_mean_data = Rating_norm(rating_table)

In [7]:

# 3. 특잇값 분해 및 행렬 축소 
# 4. Sigma로 유사행렬 도출

def Rating_svd(rating_table):
    # TruncatedSVD를 사용해서 차원축소 ( n_iter : 랜덤 SVD 계산 반복횟수 )
    svd = TruncatedSVD(n_components=12, n_iter=5)
    svd.fit(np.array(rating_table))

    # 특잇값 분해된 행렬 U, S, V
    U     = svd.fit_transform(np.array(rating_table))
    Sigma = svd.explained_variance_ratio_
    VT    = svd.components_

    print(f'추출된 sigma의 feature : \n {Sigma} \n')

    # Sigma 제곱근처리 후 유사행렬 도출
    ratings_reduced= pd.DataFrame(np.matmul(np.matmul(U, np.diag(Sigma)), VT))

    return ratings_reduced 


In [8]:
rating_table = Rating_svd(rating_table)

추출된 sigma의 feature : 
 [0.00672522 0.03242554 0.02168058 0.01787877 0.01457224 0.01213313
 0.01104605 0.0103293  0.00966095 0.00887667 0.00861798 0.00816571] 



해당 수치에 처음 뺐던 사용자 별 평균을 다시 더해주며 task 를 마친다.


In [9]:
# 5. 정규화 해제

def Rating_predict(ratings_table, row_mean_data):
    for row in range(len(ratings_table)):
        # 해당 행의 모든 컬럼 값에 행 평균 값을 더한다.
        ratings_table.iloc[row,:] = row_mean_data[row] + ratings_table.iloc[row,:]
    return ratings_table

preprocessed_rating = Rating_predict(rating_table,row_mean_data)



In [10]:
preprocessed_rating

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.095428,3.092020,3.090165,3.099352,3.087542,3.094451,3.102707,3.096840,3.098864,3.096033,...,3.088854,3.095256,3.088854,3.082452,3.088854,3.076050,3.088854,3.082452,3.088854,3.088854
1,3.085078,3.080465,3.079462,3.083997,3.081443,3.084013,3.085056,3.086362,3.086927,3.085265,...,3.079527,3.086158,3.079527,3.072896,3.079527,3.066264,3.079527,3.072896,3.079527,3.079527
2,3.062576,3.061188,3.059040,3.064006,3.060356,3.063091,3.062048,3.067489,3.065683,3.065312,...,3.060240,3.067056,3.060240,3.053424,3.060240,3.046608,3.060240,3.053424,3.060240,3.060240
3,3.097288,3.089995,3.089522,3.091918,3.091555,3.093218,3.096418,3.094705,3.094689,3.094871,...,3.088596,3.095354,3.088596,3.081839,3.088596,3.075082,3.088596,3.081839,3.088596,3.088596
4,3.035507,3.036638,3.037423,3.040466,3.036774,3.043759,3.046276,3.039644,3.041971,3.045692,...,3.038443,3.045642,3.038443,3.031243,3.038443,3.024043,3.038443,3.031243,3.038443,3.038443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,3.119458,3.110239,3.109382,3.110595,3.111461,3.110027,3.117075,3.114513,3.111747,3.111772,...,3.105951,3.112516,3.105951,3.099385,3.105951,3.092819,3.105951,3.099385,3.105951,3.105951
939,3.062814,3.059123,3.058009,3.058274,3.060507,3.061041,3.062463,3.062382,3.060294,3.062931,...,3.057481,3.064401,3.057481,3.050561,3.057481,3.043642,3.057481,3.050561,3.057481,3.057481
940,3.088983,3.081976,3.081084,3.084369,3.082396,3.084498,3.086687,3.087216,3.086492,3.086121,...,3.080023,3.086700,3.080023,3.073345,3.080023,3.066667,3.080023,3.073345,3.080023,3.080023
941,3.110992,3.101930,3.099475,3.103735,3.102124,3.102603,3.103505,3.107375,3.104566,3.103300,...,3.098774,3.105255,3.098774,3.092294,3.098774,3.085813,3.098774,3.092294,3.098774,3.098774


---

## 3. 클러스터링

In [11]:
from sklearn.cluster import KMeans

# SVD 기반 클러스터링 한 애들을 pure 한 train, test set 에 적용
def clustering(num, SVD_ratings, train_data, test_data):
    
    # 1. clusturing
    km = KMeans(n_clusters=num, init='k-means++')
    cluster = km.fit(SVD_ratings)
    cluster_id = pd.DataFrame(cluster.labels_)                   # 모든  user에 클러스터 n 이 표시됨

    cluster_id.index = SVD_ratings.index                        # userId 칼럼을 인덱스로 설정 1~943
    cluster_id.rename(columns = {0 : 'cluster'}, inplace = True) # 모든  user의 클러스터


    ## 2. train, test 에 cluster 정보 추가
    train_data = pd.concat([train_data, cluster_id], axis=1, join='inner')
    test_data = pd.concat([test_data, cluster_id], axis=1, join='inner')

    return train_data, test_data


In [12]:
train_data, test_data = clustering(6, preprocessed_rating, user_train, user_test)

#각 클러스터 당 인원수
train_data['cluster'].value_counts()

5    632
1    156
2    106
0     27
4     19
3      2
Name: cluster, dtype: int64

# 1. Average 를 통한 추천

In [30]:
# cluster 별 영화 별점 평균 행렬값 (index : cluster)
def cluster_mean(train_data):
    #train 데이터를 클러스터링으로 index 처리
    
    mean_rating = pd.DataFrame(columns = train_data.columns)
    mean_rating.set_index('cluster')
    mean_rating

    num = 6

    # 클러스터 별 사용자들의 영화 평균 별점 도출 (아무도 보지 않은 것은 Nan값 처리 된다.)
    for i in range(num):
        mean_rating = mean_rating.append(train_data[train_data.cluster == i].mean(axis=0), ignore_index=True) 

    return mean_rating.drop(['cluster'], axis=1)

mean_rating = cluster_mean(train_data)

In [31]:
# train 데이터와 test 데이터에서 겹치지 않는 아이템(영화) 를 제거한다.

def miss_matching(mean_data, test_data):

    for c in mean_data.columns:
        if c not in test_data.columns:
            del mean_data[c]
        
    for c in test_data.columns[:-1]:
        if c not in mean_data.columns:
            del test_data[c]

    return mean_data, test_data

pred_data, test_data =  miss_matching(mean_rating,test_data)

In [37]:
# top k 를 뽑을 때
pred_data[0:1].sort_values(by=0,axis=1, ascending=False)

,1175,1367,850,853,1449,169,408,318,64,12,...,1189,1306,1307,1316,1318,1334,1354,1361,1470,1500
0,5.0,5.0,5.0,4.666667,4.666667,4.546875,4.508475,4.473373,4.411392,4.409091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
print(f'예측된 클러스터링 별 item 평점과 예측할 test item은 \n{pred_data.shape}, {test_data.shape} 로 동일해졌다.')

예측된 클러스터링 별 item 평점과 예측할 test item은 
(6, 1378), (459, 1379) 로 동일해졌다.


In [16]:
#ndcg 값을 구하기 위해 추천되지 않은 아이템들은 0점을 준다.

pred_data = pred_data.fillna(0, downcast='infer')
test_data = test_data.fillna(0, downcast='infer')

- test 데이터의 마지막 열은 clustering number 을 의미한다

- 칼럼은 1682 까지 있지만 아무 클러스터도 보지 않았던 행은 자동으로 제거된다.

해당 데이터를 통해 클러스터마다 가장 높은 별점 top 5를 추천해준다.


In [17]:
from sklearn.metrics import ndcg_score

def get_NDCG(pred_data, test_data, num=6):

    #각 클러스터 별 결과
    result = [0]*num

    #각 클러스터 별 인원수
    length = list(test_data.cluster.value_counts().sort_index())

    for idx in test_data.index:

        # -1 해당 user 의 그룹을 확인
        cluster_num = int(test_data.loc[idx].cluster) 
        
        # -2 train 에서의 예측치와 test 데이터 간 각 user 별 평점 NDCG 값 계산
        result[cluster_num] += ndcg_score([test_data.loc[idx][:-1]], [pred_data.loc[cluster_num]], k=10)

        # ndcg 는 상위 k의 것을 대상으로 점수를 매길 수 있다.
        #result[cluster] += ndcg_score([user_item_test_cl.loc[idx][:-1]], [mean_rating.loc[cluster]], k=4)

    ## 5. 최종적으로 각 nDCG값 / 각 cluster의 요소 개수
    for i in range(6):
        # 클러스터 별 유저 NDCG 의 평균
        result[i] = result[i]/length[i]

    print(f"cluster 수 : {len(length)}\ncluster 별 인원 수 : {length}")
    print(f"총 NDCG : {(sum(result)/len(length)):.4f} \n\n ")

get_NDCG(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [16, 78, 48, 2, 10, 305]
총 NDCG : 0.0170 

 


In [18]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# precision & recall
def get_pre_rec(pred_data, test_data):


    # 1. 전처리
    # 평점이 2.5 이상이면 정답, 아닌 것은 오답으로 평가하였다.
    pred_data[pred_data < 2.5]= 0
    pred_data[pred_data >= 2.5]= 1

    # test 데이터 열에는 cluster 정보가 있기 때문에 백업하는 작업을 거친다.
    test_data_cluster = test_data[['cluster']].copy()
    test_data[test_data < 2.5] = 0
    test_data[test_data >= 2.5] = 1
    test_data[['cluster']] = test_data_cluster


    # 2. pre rec 계산
    pre_result = [0]*6 # 결과값 저장 리스트
    rec_result = [0]*6

    length = list(test_data.cluster.value_counts().sort_index())

    ## 4. 각 결과 값에 precision, recall더해줌
    for idx in test_data.index:
        cluster_num = int(test_data.loc[idx].cluster)
        pre_result[cluster_num] += precision_score(list(test_data.loc[idx][:-1].values) ,list(pred_data.loc[cluster_num].values ), average = 'binary')
        rec_result[cluster_num] += recall_score(list(test_data.loc[idx][:-1].values) ,list(pred_data.loc[cluster_num].values ), average = 'binary')

    for i in range(6):
        # 클러스터 별 유저 PRE/REC 의 평균
        pre_result[i] = pre_result[i]/length[i]
        rec_result[i] = rec_result[i]/length[i]

    print(f"cluster 수 : {len(length)}\ncluster 별 인원 수 : {length}")
    print(f"Presicion : {(sum(pre_result)/len(length)):.4f}")
    print(f"recall : {(sum(rec_result)/len(length)):.4f}")

In [19]:
get_pre_rec(pred_data, test_data)


cluster 수 : 6
cluster 별 인원 수 : [16, 78, 48, 2, 10, 305]
Presicion : 0.0352
recall : 0.7440


In [20]:
### top k 개 만큼 검사

from collections import defaultdict


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

## 2. Least Misery 를 통한 추천

In [21]:
train_data, test_data = clustering(6, preprocessed_rating, user_train, user_test)

#각 클러스터 당 인원수
train_data['cluster'].value_counts().sort_index()

0    632
1     19
2     27
3      2
4    106
5    156
Name: cluster, dtype: int64

In [22]:
def cluster_LM(train_data):
    #train 데이터를 클러스터링으로 index 처리
    
    mean_rating = pd.DataFrame(columns = train_data.columns)
    mean_rating.set_index('cluster')
    mean_rating

    num = 6

    # 클러스터 별 사용자들의 영화 평균 별점 도출 (아무도 보지 않은 것은 Nan값 처리 된다.)
    for i in range(num):
        mean_rating = mean_rating.append(train_data[train_data.cluster == i].min(axis=0), ignore_index=True) 

    return mean_rating.drop(['cluster'], axis=1)

LM_rating = cluster_LM(train_data)

In [23]:
pred_data, test_data =  miss_matching(LM_rating,test_data)

In [24]:
pred_data = pred_data.fillna(0)
test_data = test_data.fillna(0)

In [25]:
get_NDCG(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [305, 10, 16, 2, 48, 78]
총 NDCG : 0.0124 

 


In [26]:
get_pre_rec(pred_data, test_data)

cluster 수 : 6
cluster 별 인원 수 : [305, 10, 16, 2, 48, 78]
Presicion : 0.0254
recall : 0.2533


---

일단 행 기준으로 오름차순 정렬한 다음에, 앞에 있는 10개의 column 번호를 가져와서 추천해준다.

-> 10개의 컬럼을 가져온다.

### 특히, Average 든 LM 든

상위 10개가 판단이 되었으면 그 10개의 아이템을 가지고 다시 5개를 추려낼건데, 그 때 게임이론을 가져온다.


클러스터 내부에서 임의의 두 명. 임의의 아이템 두 개를 뽑는다. 

각 플레이어가 1,2 번 아이템을 고를지 말지를 선택하고 최적의 전략을 찾는다.

1,2, 12(두개 다선택) 중 어떤 것이 

가장 큰 효율을 내는지 찾아낸다.

각 클러스터 인원의 10분의 1만큼 전략을 찾아서 카운팅을 한다.

1,2, 12 중 하나를 고른다.

---
(12) 가 가장 많이 골라졌다고 가정했을때,

12, 3, 123번을 고를지 말지 선택하는 전략을 찾아서 카운팅을 한다.
---
반복하다보면

걸러내야 할 것을 찾을 수 있다. 10개 중 12678 처럼 아이템이 다시 추려질 것이다.

이것이 10개중 상위 5개보다 이상적인지를 체크한다.

In [27]:
#LM_rating.iloc[0]
LM_rating.iloc

In [29]:
LM_rating[0:1].sort_values(by=0, axis=1)

,1,755,756,758,759,760,761,762,763,764,...,1189,1306,1307,1316,1318,1334,1354,1361,1470,1500
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
